In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['LSK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('lin sca 1 + c kit +', 19),
 ('lin − sca1 + c kit +', 15),
 ('lineage sca 1 + c kit +', 9),
 ('lin − sca1 + ckit +', 9),
 ('lin − sca 1 + ckit +', 7),
 ('lowsca 1 + c kit +', 6),
 ('lin − sca 1 + kit +', 5),
 ('lin sca 1 + kit +', 4),
 ('lin − sca + ckit +', 4),
 ('sca 1 + c kit + cells', 4),
 ('lin sca1 + c kit +', 3)]

In [10]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [11]:
result = [grounding_map, names, pos_labels]

In [12]:
result

[{'lin sca 1 + c kit +': 'CL:CL:0001025',
  'lin sca 1 + kit +': 'CL:CL:0001025',
  'lin sca1 + c kit +': 'CL:CL:0001025',
  'lin − sca + ckit +': 'CL:CL:0001025',
  'lin − sca 1 + ckit +': 'CL:CL:0001025',
  'lin − sca 1 + kit +': 'CL:CL:0001025',
  'lin − sca1 + c kit +': 'CL:CL:0001025',
  'lin − sca1 + ckit +': 'CL:CL:0001025',
  'lineage sca 1 + c kit +': 'CL:CL:0001025',
  'lowsca 1 + c kit +': 'CL:CL:0001025',
  'sca 1 + c kit + cells': 'CL:CL:0001025'},
 {'CL:CL:0001025': 'Kit-positive, Sca1-positive common lymphoid progenitor'},
 ['CL:CL:0001025']]

In [13]:
grounding_map, names, pos_labels = [{'lin sca 1 + c kit +': 'CL:CL:0001025',
  'lin sca 1 + kit +': 'CL:CL:0001025',
  'lin sca1 + c kit +': 'CL:CL:0001025',
  'lin − sca + ckit +': 'CL:CL:0001025',
  'lin − sca 1 + ckit +': 'CL:CL:0001025',
  'lin − sca 1 + kit +': 'CL:CL:0001025',
  'lin − sca1 + c kit +': 'CL:CL:0001025',
  'lin − sca1 + ckit +': 'CL:CL:0001025',
  'lineage sca 1 + c kit +': 'CL:CL:0001025',
  'lowsca 1 + c kit +': 'CL:CL:0001025',
  'sca 1 + c kit + cells': 'CL:CL:0001025'},
 {'CL:CL:0001025': 'Kit-positive, Sca1-positive common lymphoid progenitor'},
 ['CL:CL:0001025']]

In [14]:
excluded_longforms = []

In [15]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [16]:
additional_entities = {'HGNC:6524': ['LCK', ['LCK']]}

In [17]:
unambiguous_agent_texts = {}

In [18]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [19]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [20]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [21]:
intersection1

[('HGNC:6524', 'HGNC:6524', 525)]

In [22]:
intersection2

[('CL:CL:0001025', 'HGNC:6524', 0)]

In [23]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [24]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [25]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 03:46:05] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 03:46:07] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9966246040503688 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [26]:
classifier.stats

{'label_distribution': {'CL:CL:0001025': 67, 'HGNC:6524': 525},
 'f1': {'mean': 0.996625, 'std': 0.004134},
 'precision': {'mean': 0.996761, 'std': 0.003969},
 'recall': {'mean': 0.996624, 'std': 0.004134},
 'HGNC:6524': {'f1': {'mean': 0.998095, 'std': 0.002333},
  'pr': {'mean': 0.998095, 'std': 0.00381},
  'rc': {'mean': 0.998113, 'std': 0.003774}},
 'CL:CL:0001025': {'f1': {'mean': 0.985185, 'std': 0.018144},
  'pr': {'mean': 0.985714, 'std': 0.028571},
  'rc': {'mean': 0.985714, 'std': 0.028571}}}

In [27]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [28]:
disamb.dump(model_name, results_path)

In [29]:
print(disamb.info())

Disambiguation model for LSK

Produces the disambiguations:
	Kit-positive, Sca1-positive common lymphoid progenitor*	CL:CL:0001025
	LCK*	HGNC:6524

Class level metrics:
--------------------
Grounding                                             	Count	F1     
                                                   LCK*	525	 0.9981
Kit-positive, Sca1-positive common lymphoid progenitor*	 67	0.98518

Weighted Metrics:
-----------------
	F1 score:	0.99662
	Precision:	0.99676
	Recall:		0.99662

* Positive labels
See Docstring for explanation



In [25]:
model_to_s3(disamb)

In [30]:
preds = disamb.disambiguate(all_texts.values())

In [32]:
texts = [text for text, pred in zip(all_texts.values(), preds) if pred[0] == 'HGNC:6524']

In [34]:
print('\n\n'.join(texts))

Gfi1 is required for adult hematopoietic stem cell (HSC) quiescence [18,19]. Deregulation of the Gfi1 target gene p21cip1/waf[20] was posited as a partial mechanism to explain the loss of HSC quiescence [18,19]. However, Gfi1−/− and p21−/− display dissimilar lineage negative, Sca1+, cKit+(LSK) numbers, with Gfi1−/− stem/progenitors in active cell cycle leading to decreasing numbers of LSK while p21−/− induces a decrease in stem/progenitor G2-M phase [18,19,21-23]. In fact, recent studies have demonstrated that HSC quiescence is a p21-independent function of p53 [24]. Notably, while deregulated p21Waf1 is not sufficient to phenocopy p53−/− or Gfi1−/− HSC effects, it is formally possible that deregulation might still be a necessary precursor to p53−/− or Gfi1−/− HSC phenotypes.
Recently, p53 and the Ets transcription factor Mef/Elf4 have been implicated in controlling HSC maintenance, perhaps through Gfi1 (Fig 1) [24]. In the absence of Mef/Elf4, a known regulator of HSC [25], LSK displa